In [5]:
import cv2
import dlib
import numpy as np

# DNNベースの顔検出器のロード
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 特定領域の処理を行うための関数
def apply_transformation(roi, fx=1.0, fy=1.0):
    # 領域を指定された倍率でリサイズ
    resized_roi = cv2.resize(roi, None, fx=fx, fy=fy)
    return resized_roi

# カメラの指定
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_detector(gray)
    
    for face in faces:
        # 顔のランドマークを取得
        landmarks = landmark_predictor(gray, face)

        # --- 各領域の分割処理 ---

        # 1. 目の領域の処理（目を大きくする）
        left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
        right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])

        # 左目の領域を抽出し拡大
        left_eye_rect = cv2.boundingRect(left_eye_points)
        x, y, w, h = left_eye_rect
        left_eye_roi = frame[y:y + h, x:x + w]
        big_left_eye = apply_transformation(left_eye_roi, fx=1.2, fy=1.2)  # 目を1.2倍に拡大
        new_height, new_width = big_left_eye.shape[:2]
        frame[y:y + new_height, x:x + new_width] = big_left_eye

        # 右目の領域を抽出し拡大
        right_eye_rect = cv2.boundingRect(right_eye_points)
        x, y, w, h = right_eye_rect
        right_eye_roi = frame[y:y + h, x:x + w]
        big_right_eye = apply_transformation(right_eye_roi, fx=1.2, fy=1.2)  # 目を1.2倍に拡大
        new_height, new_width = big_right_eye.shape[:2]
        frame[y:y + new_height, x:x + new_width] = big_right_eye

        # 2. 鼻の領域の処理（鼻を高くする）
        nose_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(27, 36)])
        nose_rect = cv2.boundingRect(nose_points)
        x, y, w, h = nose_rect
        nose_roi = frame[y:y + h, x:x + w]
        high_nose = apply_transformation(nose_roi, fx=1.0, fy=1.2)  # 縦方向に1.2倍に拡大
        new_height, new_width = high_nose.shape[:2]
        frame[y:y + new_height, x:x + new_width] = high_nose

        # 3. 口の領域の処理（口を強調する）
        mouth_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)])
        mouth_rect = cv2.boundingRect(mouth_points)
        x, y, w, h = mouth_rect
        mouth_roi = frame[y:y + h, x:x + w]
        big_mouth = apply_transformation(mouth_roi, fx=1.2, fy=1.2)  # 口を1.2倍に拡大
        new_height, new_width = big_mouth.shape[:2]
        frame[y:y + new_height, x:x + new_width] = big_mouth

    # フレームを表示
    cv2.imshow('Feature-based Face Modification', frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import dlib
import time
import os

# DNNベースの顔検出器のロード
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# カウントダウン秒数
COUNTDOWN_SECONDS = 5

# カメラの指定
cap = cv2.VideoCapture(0)

# シャッター音の再生（OpenCVは音を扱えないため、winsoundやpygameを使用するか、代わりに効果音を表示）
def play_shutter_sound():
    # ここでシャッター音を再生するロジックを追加します
    # pygame.mixerなどを使ってシャッター音ファイルを再生することが可能です
    print("シャッター音が再生されました")  # 実際には音を再生します

# スタートボタンが押されると、カウントダウンを開始
def start_countdown():
    print("スタートボタンが押されました！")
    for i in range(COUNTDOWN_SECONDS, 0, -1):
        print(f"撮影まで: {i}秒")
        time.sleep(1)

# 画像を保存する関数
def save_image(frame):
    filename = f'photo_{time.strftime("%Y%m%d_%H%M%S")}.png'
    cv2.imwrite(filename, frame)
    print(f"画像が保存されました: {filename}")

# メインループ
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_detector(gray)
    
    # 検出された顔にランドマークを描画（例: 目や口）
    for face in faces:
        landmarks = landmark_predictor(gray, face)
        for n in range(68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)

    # フレームを表示
    cv2.imshow('Preview', frame)

    # 's'キーでスタートボタンの役割
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # スタートボタンを押すとカウントダウン開始
        start_countdown()

        # カウントダウンが終わったらシャッター音を再生し、写真を保存
        play_shutter_sound()
        save_image(frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()


スタートボタンが押されました！
撮影まで: 5秒
撮影まで: 4秒
撮影まで: 3秒
撮影まで: 2秒
撮影まで: 1秒
シャッター音が再生されました
画像が保存されました: photo_20241001_163651.png
